In [1]:
# In this example we show the use of general constraints for modeling
# some common expressions . We use as an example a SAT - problem where we
# want to see if it is possible to satisfy at least four (or all) clauses
# of the logical for
#
# L = (x0 or ~x1 or x2)   and (x1 or ~x2 or x3)   and
#     (x2 or ~x3 or x0)   and (x3 or ~x0 or x1)   and
#     (~ x0 or ~x1 or x2) and (~ x1 or ~x2 or x3) and
#     (~ x2 or ~x3 or x0) and (~ x3 or ~x0 or x1)
#
# We do this by introducing two variables for each literal ( itself and its
# negated value ), a variable for each clause , and then two
# variables for indicating if we can satisfy four , and another to identify
# the minimum of the clauses (so if it is one , we can satisfy all clauses )
# and put these two variables in the objective .
# i.e. the Objective function will be
#
# maximize Obj0 + Obj1
#
# Obj0 = MIN( Clause1 , ... , Clause8 )
# Obj1 = 1 -> Clause1 + ... + Clause8 >= 4
#
# thus , the objective value will be two if and only if we can satisfy all
# clauses ; one if and only if at least four clauses can be satisfied , and
# zero otherwise .

In [2]:
import gurobipy as gp
from gurobipy import GRB
import sys

In [3]:
NLITERALS = 4
n = NLITERALS

# Example data:
# e.g. {0, n+1, 2} means clause (x0 or ~x1 or x2)
Clauses = [
    [0, n+1, 2],
    [1, n+2, 3],
    [2, n+3, 0],
    [3, n+0, 1],
    [n+0, n+1, 2],
    [n+1, n+2, 3],
    [n+2, n+3, 0],
    [n+3, n+0, 1]
]

In [4]:
# Create a new model
model = gp.Model('Genconstr')

Using license file /Users/yj/gurobi.lic


In [5]:
# initialize decision variable and objective
Lit = model.addVars(NLITERALS, vtype=GRB.BINARY, name='X')
NotLit = model.addVars(NLITERALS, vtype=GRB.BINARY, name='NotX')

Cla = model.addVars(len(Clauses), vtype=GRB.BINARY, name='Clause')

Obj0 = model.addVar(vtype=GRB.BINARY, name='Obj1')
Obj1 = model.addVar(vtype=GRB.BINARY, name='Obj2')

In [11]:
# Link Xi and notXi
model.addConstrs((Lit[i] + NotLit[i] == 1.0 for i in range(NLITERALS)),
                 name='CNSTR_X')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [13]:
# Link clauses and literals
for i, c  in enumerate(Clauses):
    # print(i,c)
    clause = []
    for l in c:
        if l >= n:
            clause.append(NotLit[l-n])
        else:
            clause.append(Lit[l])
    model.addConstr(Cla[i] == gp.or_(clause), 'CNSTR_Clause' + str(i))

In [14]:
# Link objs with clauses
model.addConstr(Obj0 == gp.min_(Cla), name = 'CNSTR_Obj0')
model.addConstr((Obj1 == 1) >> (Cla.sum() >= 4.0), name = 'CNSTR_Obj1')

<gurobi.GenConstr *Awaiting Model Update*>

In [15]:
# Set optimization objective
model.setObjective(Obj0 + Obj1, GRB.MAXIMIZE)

In [16]:
# Save problem
model.write('genconstr.mps')
model.write('genconstr.lp')

In [17]:
# Optimize
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 4 rows, 18 columns and 8 nonzeros
Model fingerprint: 0x14d622b5
Model has 10 general constraints
Variable types: 0 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve added 37 rows and 0 columns
Presolve removed 0 rows and 4 columns
Presolve time: 0.04s
Presolved: 41 rows, 14 columns, 105 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)
Found heuristic solution: objective 1.0000000
Found heuristic solution: objective 2.0000000

Explored 0 nodes (0 simplex iterations) in 0.16 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 2 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%


In [18]:
# Status checking
status = model.getAttr(GRB.Attr.Status)

In [19]:
if status in (GRB. INF_OR_UNBD, GRB. INFEASIBLE, GRB. UNBOUNDED ):
    print ("The model cannot be solved because it is infeasible or unbounded ")
    sys. exit (1)

if status != GRB . OPTIMAL :
    print ("Optimization was stopped with status", status)
    sys. exit (1)

In [21]:
# Print result
objval = model.getAttr(GRB.Attr.ObjVal)

if objval > 1.9:
    print (" Logical expression is satisfiable ")
elif objval > 0.9:
    print ("At least four clauses can be satisfied ")
else :
    print ("Not even three clauses can be satisfied ")

 Logical expression is satisfiable 
